# Комп'ютерний практикум №3
## Сидорський Володимир Сергійович (КА-21ф)
## Варіант 5

# Завдання

![Task_main](images/task_1.png)


![Task_input](images/task_2.png)

# Програма

In [1]:
import numpy as np
import pandas as pd

In [2]:
def rgmm(D):
    w = np.prod(D, axis=1) ** (1/D.shape[0])
    return w / w.sum(), {"or_w": w}

def compute_gci(D, or_w):
    n = D.shape[0]
    gci = 0
    # Sum only elements where i < j
    for i in range(n):
        for j in range(i + 1, n):
            gci += (np.log(D[i][j]) - np.log(or_w[i] / or_w[j])) ** 2
            
    gci *= (2 / (n -1) * (n-2))

    # Matrix dims: 0, 1, 2, 3, 4, 5
    THRESHS = [0, 0, 0, 0.1573, 0.3526, 0.370]
    thresh = THRESHS[n]

    if gci > thresh:
        print(f"GCI(={gci:.3f}) > {thresh} - МПП неузгоджена")
    elif gci > 0:
        print(f"GCI(={gci:.3f}) <= {thresh} - МПП допустимо узгоджена")
    else:
        print(f"GCI = 0.0 - МПП узгоджена")

In [3]:
def check_fuzy_consistency(mpp):
    n = mpp.shape[0]  # Assuming L and U are square matrices of the same size
    consistent = True
    for i in range(n):
        for j in range(n):
            max_l = max(mpp[i, k, 0] * mpp[k, j, 0] for k in range(n))
            min_u = min(mpp[i, k, -1] * mpp[k, j, -1] for k in range(n))
            if max_l > min_u:
                consistent = False
                print(f"НМПП неузгоджена. Бо max_k(l_{i+1}k * l_k{j+1}) <= min_k(u_{i+1}k * u_k{j+1})")
                break
        if not consistent:
            break
    if consistent:
        print("НМПП узгоджена")

def frgmm(D):
    w = np.prod(D, axis=1) ** (1/D.shape[0])
    return w / w.sum(axis=0)[::-1], {"or_w": w}

def v_func_s_i_dom_s_j(fuz_i, fuz_j):
    if fuz_i[1] >= fuz_j[1]:
        return 1
    elif fuz_j[0] <= fuz_i[-1] and fuz_i[1] < fuz_j[1]:
        return (fuz_i[-1] - fuz_j[0]) / ((fuz_i[-1] - fuz_i[1]) + (fuz_j[1] - fuz_j[0]))
    elif fuz_j[0] > fuz_i[-1]:
        return 0
    else:
        raise Exception(f"No matching condition for fuz_i({fuz_i}) and fuz_j({fuz_j})")

def fahp(D):
    RS = D.sum(axis=1)
    S = RS / RS[:,::-1].sum(axis=0)
    V = np.zeros(D.shape[:-1])
    V_si = np.array([[v_func_s_i_dom_s_j(S[i], S[j]) for j in range(len(S))] for i in range(len(S))])
    V_si = V_si.min(axis=1)
    return V_si / V_si.sum()
    

def rank(weights):
    n = weights.shape[0]
    P = np.empty((n, n))
    for i in range(n):
        for j in range(n):
            a_l, a_u = weights[i,0], weights[i,-1]
            b_l, b_u = weights[j,0], weights[j,-1]
            P[i, j] = (max(a_u - b_l, 0) - max(a_l - b_u, 0)) / ((a_u - b_l) - (a_l - b_u))

    ranking = (-P.sum(1)).argsort()
    return list(ranking)

# Виклик програми

## Перевіримо алгоритми на матриці з лекції 15-17

Вхідна матриця `A`

In [4]:
l_matrix = np.array([
    [1, 4, 6, 1, 7, 2, 5, 3],
    [0.167, 1, 3, 0.2, 5, 0.25, 2, 0.333],
    [0.125, 0.2, 1, 0.143, 2, 0.167, 0.333, 0.2],
    [0.333, 3, 5, 1, 6, 1, 4, 2],
    [0.111, 0.143, 0.25, 0.125, 1, 0.143, 0.2, 0.143],
    [0.25, 2, 4, 0.333, 5, 1, 3, 1],
    [0.143, 0.25, 1, 0.167, 3, 0.2, 1, 0.25],
    [0.2, 1, 3, 0.25, 5, 0.333, 2, 1]
])
m_matrix = np.array([
    [1, 5, 7, 2, 8, 3, 6, 4],
    [1/5, 1, 4, 1/4, 6, 1/3, 3, 1/2],
    [1/7, 1/4, 1, 1/6, 3, 1/5, 1/2, 1/4],
    [1/2, 4, 6, 1, 7, 2, 5, 3],
    [1/4, 1/6, 1/3, 1/7, 1, 1/6, 1/4, 1/6],
    [1/3, 3, 5, 1/2, 6, 1, 4, 2],
    [1/6, 1/3, 2, 1/5, 4, 1/4, 1, 1/3],
    [1/4, 2, 4, 1/3, 6, 1/2, 3, 1]
])
u_matrix = np.array([
    [1, 6, 8, 3, 9, 4, 7, 5],
    [1/4, 1, 5, 1/3, 7, 1/2, 4, 1],
    [1/6, 1/3, 1, 1/5, 4, 1/4, 1, 1/3],
    [1, 5, 7, 1, 8, 3, 6, 4],
    [1/7, 1/5, 1/2, 1/6, 1, 1/5, 1/3, 1/5],
    [1/2, 4, 6, 1, 7, 1, 5, 3],
    [1/5, 1/2, 3, 1/4, 5, 1/3, 1, 1/2],
    [1/3, 3, 5, 1/2, 7, 1, 4, 1]
])

A_triang = np.stack((l_matrix,m_matrix,u_matrix), axis=-1)
A_triang

array([[[1.        , 1.        , 1.        ],
        [4.        , 5.        , 6.        ],
        [6.        , 7.        , 8.        ],
        [1.        , 2.        , 3.        ],
        [7.        , 8.        , 9.        ],
        [2.        , 3.        , 4.        ],
        [5.        , 6.        , 7.        ],
        [3.        , 4.        , 5.        ]],

       [[0.167     , 0.2       , 0.25      ],
        [1.        , 1.        , 1.        ],
        [3.        , 4.        , 5.        ],
        [0.2       , 0.25      , 0.33333333],
        [5.        , 6.        , 7.        ],
        [0.25      , 0.33333333, 0.5       ],
        [2.        , 3.        , 4.        ],
        [0.333     , 0.5       , 1.        ]],

       [[0.125     , 0.14285714, 0.16666667],
        [0.2       , 0.25      , 0.33333333],
        [1.        , 1.        , 1.        ],
        [0.143     , 0.16666667, 0.2       ],
        [2.        , 3.        , 4.        ],
        [0.167     , 0.2      

Перевіряємо узгодженість

In [5]:
check_fuzy_consistency(A_triang)

НМПП неузгоджена. Бо max_k(l_1k * l_k2) <= min_k(u_1k * u_k2)


Використаємо метод `FRGMM` для підрахунку нечітких ваг і метод `Ступення переваги` для ранжування

In [6]:
frgmm_weights, add_outputs = frgmm(A_triang)

In [7]:
for idx, weight in enumerate(frgmm_weights, start=1):
    print(f"w{idx}", weight)

w1 [0.19532499 0.32439208 0.51699984]
w2 [0.04932993 0.08084424 0.14186928]
w3 [0.02067836 0.03285483 0.0565234 ]
w4 [0.13348304 0.22937984 0.39283493]
w5 [0.0130724  0.02127066 0.03117884]
w6 [0.08972215 0.15677537 0.27777624]
w7 [0.02798752 0.04667544 0.08046996]
w8 [0.06169798 0.10780753 0.19354682]


In [8]:
ranked_alternatives = rank(frgmm_weights)

In [9]:
print("Відсортовані (від кращої до гіршої) альтернативи")
print([f"a{idx+1}" for idx in ranked_alternatives])

Відсортовані (від кращої до гіршої) альтернативи
['a1', 'a4', 'a6', 'a8', 'a2', 'a7', 'a3', 'a5']


Використаємо метод `FAHP` для підрахунку ваг

In [10]:
fahp_weights = fahp(A_triang)

In [11]:
for idx, weight in enumerate(fahp_weights, start=1):
    print(f"w{idx}", weight)

w1 0.40810242421834636
w2 0.021243492336286
w3 0.0
w4 0.30295214022655736
w5 0.0
w6 0.1868384610485808
w7 0.0
w8 0.08086348217022961


In [12]:
print("Відсортовані (від кращої до гіршої) альтернативи. Альтернативи які мають однакову вагу сортуются за індексом")
print([f"a{idx+1}" for idx in np.argsort(-fahp_weights)])

Відсортовані (від кращої до гіршої) альтернативи. Альтернативи які мають однакову вагу сортуются за індексом
['a1', 'a4', 'a6', 'a8', 'a2', 'a3', 'a5', 'a7']


Звіримо з результатами `Lection_ISPPR_15-17_fuzzy_DSS_1.pdf` ст.27

![Task_Ref](images/task_ref.png)

Значення ваг збігається

## Застосуємо алгоритми на матрицю з завдання

In [13]:
A = np.array([
    [[1,1], [1,1], [1/6, 1/5], [1/4, 1/3]],
    [[1,1], [1,1], [1/6, 1/5], [1/4, 1/3]],
    [[5, 6], [5, 6], [1,1], [4, 5]],
    [[3, 4], [3, 4], [1/5, 1/4], [1,1]],
])
A

array([[[1.        , 1.        ],
        [1.        , 1.        ],
        [0.16666667, 0.2       ],
        [0.25      , 0.33333333]],

       [[1.        , 1.        ],
        [1.        , 1.        ],
        [0.16666667, 0.2       ],
        [0.25      , 0.33333333]],

       [[5.        , 6.        ],
        [5.        , 6.        ],
        [1.        , 1.        ],
        [4.        , 5.        ]],

       [[3.        , 4.        ],
        [3.        , 4.        ],
        [0.2       , 0.25      ],
        [1.        , 1.        ]]])

Матриця задана у вигляді 

$$d_{ij} = (l_{ij}, u_{ij})$$

Проте для застосування попередніх методів необхіді нечіткі трикутні числа

$$d_{ij} = (l_{ij}, m_{ij}, u_{ij})$$

Покладемо

$$m_{ij} = (l_{ij} + u_{ij}) / 2$$

In [14]:
A_triang = np.array([
    [[1,1,1], [1,1,1], [1/6, (1/5.5), 1/5], [1/4, (1/3.5), 1/3]],
    [[1,1,1], [1,1,1], [1/6, (1/5.5), 1/5], [1/4, (1/3.5), 1/3]],
    [[5, 5.5, 6], [5, 5.5, 6], [1,1,1], [4, 4.5, 5]],
    [[3, 3.5, 4], [3, 3.5, 4], [1/5, (1/4.5), 1/4], [1,1,1]],
])
A_triang

array([[[1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [0.16666667, 0.18181818, 0.2       ],
        [0.25      , 0.28571429, 0.33333333]],

       [[1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [0.16666667, 0.18181818, 0.2       ],
        [0.25      , 0.28571429, 0.33333333]],

       [[5.        , 5.5       , 6.        ],
        [5.        , 5.5       , 6.        ],
        [1.        , 1.        , 1.        ],
        [4.        , 4.5       , 5.        ]],

       [[3.        , 3.5       , 4.        ],
        [3.        , 3.5       , 4.        ],
        [0.2       , 0.22222222, 0.25      ],
        [1.        , 1.        , 1.        ]]])

Перевіряємо узгодженість

In [15]:
check_fuzy_consistency(A_triang)

НМПП неузгоджена. Бо max_k(l_1k * l_k3) <= min_k(u_1k * u_k3)


Використаємо метод `FRGMM` для підрахунку нечітких ваг і метод `Ступення переваги` для ранжування

In [16]:
frgmm_weights, add_outputs = frgmm(A_triang)

In [17]:
for idx, weight in enumerate(frgmm_weights, start=1):
    print(f"w{idx}", weight)

w1 [0.07414693 0.08442205 0.0972657 ]
w2 [0.07414693 0.08442205 0.0972657 ]
w3 [0.51897445 0.60401546 0.70113343]
w4 [0.19009211 0.22714044 0.27070579]


In [18]:
ranked_alternatives = rank(frgmm_weights)

In [19]:
print("Відсортовані (від кращої до гіршої) альтернативи")
print([f"a{idx+1}" for idx in ranked_alternatives])

Відсортовані (від кращої до гіршої) альтернативи
['a3', 'a4', 'a1', 'a2']


Використаємо метод `FAHP` для підрахунку ваг

In [20]:
fahp_weights = fahp(A_triang)

In [21]:
for idx, weight in enumerate(fahp_weights, start=1):
    print(f"w{idx}", weight)

w1 0.0
w2 0.0
w3 1.0
w4 0.0


In [22]:
print("Відсортовані (від кращої до гіршої) альтернативи. Альтернативи які мають однакову вагу сортуются за індексом")
print([f"a{idx+1}" for idx in np.argsort(-fahp_weights)])

Відсортовані (від кращої до гіршої) альтернативи. Альтернативи які мають однакову вагу сортуются за індексом
['a3', 'a1', 'a2', 'a4']


Перейдемо до чіткої МПП, елементи якої – це середнє геометричне значення кінців інтервалів заданої ІМПП

In [23]:
A_nf = np.prod(A, axis=-1) ** (1/A.shape[-1])
A_nf

array([[1.        , 1.        , 0.18257419, 0.28867513],
       [1.        , 1.        , 0.18257419, 0.28867513],
       [5.47722558, 5.47722558, 1.        , 4.47213595],
       [3.46410162, 3.46410162, 0.2236068 , 1.        ]])

![Task_Prev](images/task_lab_1.png)

Знайдемо ваги альтернатив методом `RGMM` та дослідимо узгодженість за допомгою індексу `GCI`

In [24]:
rgmm_weights, add_outputs = rgmm(A_nf)

In [25]:
for idx, weight in enumerate(rgmm_weights, start=1):
    print(f"w{idx}", weight)

w1 0.08493095335244578
w2 0.08493095335244578
w3 0.6032715918622685
w4 0.2268665014328399


In [26]:
compute_gci(A_nf, **add_outputs)

GCI(=0.721) > 0.3526 - МПП неузгоджена


In [27]:
print("Відсортовані (від кращої до гіршої) альтернативи")
print([f"a{idx+1}" for idx in np.argsort(-rgmm_weights)])

Відсортовані (від кращої до гіршої) альтернативи
['a3', 'a4', 'a1', 'a2']


Ранжування в чіткому випадку співпадає з ранжування в нечіткому випадку з точністю до альтернатив з ненульовими вагами, отриманих з методу `FAHP`

# Висновки

В даній роботі було досліджено метод розрахунку ваг альтернатив для нечітких МПП. Був використаний метод `FRGMM` для підрахунку нечітких ваг, а потім метод `Ступеня перваги` для ранжування альтернатив. Також був використаний метод `FAHP` для підрахунку чітких ваг. Потім з нечіткої МПП було отримано чітку й використано метод `RGMM` для підрахунку ваг. Ранжування в чіткому випадку співпадає з ранжування в нечіткому випадку з точністю до альтернатив з ненульовими вагами, отриманих з методу `FAHP`. Нечітка МПП була досліджена на узогдженість за критерієм, а чітка за індексом `GCI`. Всі використані методи було запрограмовано, текст програми наведено в звіті.